In [1]:
pip install cassandra-driver clickhouse-connect pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Ejecuta esta celda para instalar las librerías faltantes
!pip install cassandra-driver pandas numpy

In [3]:
import sys
import pandas as pd
import numpy as np
from cassandra.cluster import Cluster
from uuid import uuid4
from datetime import datetime, timedelta

# Solución de compatibilidad (Mock asyncore) para Python 3.13+
if 'asyncore' not in sys.modules:
    import types
    mock_asyncore = types.ModuleType('asyncore')
    sys.modules['asyncore'] = mock_asyncore

# 1. Conexión al clúster de Cassandra
# Usamos el nombre del servicio definido en el docker-compose
cluster = Cluster(['cassandra_db'])
session = cluster.connect('proyecto_bigdata')

# 2. Generación de datos (100,000 registros)
print("Generando datos...")
n = 100000
categorias = ['Electrónica', 'Ropa', 'Hogar', 'Alimentos', 'Deportes']

# Preparamos la consulta para mejorar el rendimiento de inserción
query = session.prepare("""
    INSERT INTO ventas_crudas (id_venta, fecha_venta, id_producto, categoria, monto_total, id_cliente)
    VALUES (?, ?, ?, ?, ?, ?)
""")

# 3. Inserción Masiva
print("Iniciando ingesta masiva en Cassandra...")
for i in range(n):
    session.execute(query, (
        uuid4(), 
        datetime.now().date() - timedelta(days=np.random.randint(0, 60)),
        f"PROD-{np.random.randint(100, 999)}",
        np.random.choice(categorias),
        float(np.round(np.random.uniform(5.0, 1000.0), 2)),
        f"CLI-{np.random.randint(1000, 5000)}"
    ))

print("¡Ingesta completada exitosamente!")

Generando datos...
Iniciando ingesta masiva en Cassandra...
¡Ingesta completada exitosamente!


In [4]:
import sys
from cassandra.cluster import Cluster
from uuid import uuid4
from datetime import datetime, timedelta
import numpy as np

# 1. Conexión con reintentos
try:
    cluster = Cluster(['cassandra_db']) 
    session = cluster.connect('proyecto_bigdata')
    print("✅ Conexión establecida con Cassandra")
except Exception as e:
    print(f"❌ Error de conexión: {e}")
    print("Tip: Asegúrate de que Cassandra terminó de subir y creaste el keyspace.")

# 2. Configuración de Ingesta (100,000 registros)
n_registros = 100000
categorias = ['Electrónica', 'Ropa', 'Hogar', 'Alimentos', 'Deportes']

# Preparar la query (Batch implícito)
query = session.prepare("""
    INSERT INTO ventas_crudas (id_venta, fecha_venta, id_producto, categoria, monto_total, id_cliente)
    VALUES (?, ?, ?, ?, ?, ?)
""")

print(f"🚀 Iniciando ingesta de {n_registros} registros...")

# 3. Inserción (Tarea 2.2)
for i in range(n_registros):
    session.execute(query, (
        uuid4(), 
        datetime.now().date() - timedelta(days=np.random.randint(0, 60)),
        f"PROD-{np.random.randint(100, 999)}",
        np.random.choice(categorias),
        float(np.round(np.random.uniform(5.0, 1000.0), 2)),
        f"CLI-{np.random.randint(1000, 5000)}"
    ))
    if i % 10000 == 0:
        print(f"Procesados: {i} registros...")

print("✨ Ingesta masiva completada exitosamente.")

✅ Conexión establecida con Cassandra
🚀 Iniciando ingesta de 100000 registros...
Procesados: 0 registros...


KeyboardInterrupt: 